In [1]:
# Torch
import torch
import torch.optim as optim
from torcheval.metrics import *

import pickle
from captum.attr import *
import random
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

np.set_printoptions(threshold=np.inf)

# Custom modules
from preprocessing_post_fastsurfer.subject import *
from preprocessing_post_fastsurfer.vis import *
from ozzy_torch_utils.split_dataset import *
from ozzy_torch_utils.subject_dataset import *
from ozzy_torch_utils.plot import *
from ozzy_torch_utils.train_nn import *
from ozzy_torch_utils.model_parameters import *
from ozzy_torch_utils.init_dataloaders import *
from explain_pointnet import *

In [2]:
# Load dataset
data_path = "/uolstore/home/users/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/scratch-disk/full-datasets/hcampus-1.5T-cohort"

subject_list = find_subjects_parallel(data_path)

Csv files: ['/uolstore/home/users/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/scratch-disk/full-datasets/hcampus-1.5T-cohort/idaSearch_3_19_2025.csv']


In [3]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

Checking the significance of left and right hippocampi on models run with both

In [4]:
subject = sample(subject_list, 1)[0]

In [5]:
# Load model
pickle_pathname = "/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/runs/run_18-03-2025_18-04-04/run_18-03-2025_18-04-04_params.pkl"

with open(pickle_pathname, 'rb') as file:
    
    model_parameters = pickle.load(file)
    
model = model_parameters.model

In [6]:
lr_cloud = np.load(os.path.join(subject.path, "Left-Hippocampus_Right-Hippocampus_aligned_cropped_mesh_downsampledcloud.npy"))

lr_attributions, lr_pred_research_group = pointnet_ig(model, lr_cloud, device)

vis_attributions(lr_attributions, subject, lr_cloud, lr_pred_research_group)

Widget(value='<iframe src="http://localhost:34927/index.html?ui=P_0x7f414e0e3020_0&reconnect=auto" class="pyvi…

In [7]:
# Load model
pickle_pathname = "/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/runs/run_19-03-2025_16-12-19/run_19-03-2025_16-12-19_params.pkl"

with open(pickle_pathname, 'rb') as file:
    
    model_parameters = pickle.load(file)
    
model = model_parameters.model

In [8]:
l_cloud = np.load(os.path.join(subject.path, "Left-Hippocampus_aligned_cropped_mesh_downsampledcloud.npy"))

l_attributions, l_pred_research_group = pointnet_ig(model, l_cloud, device)

vis_attributions(l_attributions, subject, l_cloud, l_pred_research_group)

Widget(value='<iframe src="http://localhost:34927/index.html?ui=P_0x7f405c56d670_1&reconnect=auto" class="pyvi…

Interesting experiment comparing attributions from two permutations of the same cloud

In [9]:
# Load model
pickle_pathname = "/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/runs/run_18-03-2025_15-35-05/run_18-03-2025_15-35-05_params.pkl"

with open(pickle_pathname, 'rb') as file:
    
    model_parameters = pickle.load(file)
    
model = model_parameters.model

In [10]:
cloud = np.load(os.path.join(subject.path, "Left-Hippocampus_aligned_cropped_mesh_downsampledcloud.npy"))
    

In [11]:
attributions_orig, pred_research_group_orig = pointnet_ig(model, cloud, device)

shuffler = np.random.permutation(cloud.shape[0])

unshuffler = np.argsort(shuffler)

cloud_shuffled = np.array([cloud[i] for i in shuffler])

attributions_shuffled, pred_research_group_shuffle = pointnet_ig(model, cloud_shuffled, device)

cloud_unshuffled = np.array([cloud_shuffled[i] for i in unshuffler])

attributions_unshuffled = np.array([attributions_shuffled[i] for i in unshuffler])

attributions_diff = attributions_orig - attributions_unshuffled

# NB can't really visualise attributions as they will be normalsied and look large
print(attributions_diff)

if pred_research_group_orig != pred_research_group_shuffle:
    
    print("Research groups are different after shuffle")

print(attributions_diff.shape)
    

[[-1.24690454e-07  1.88965682e-07  9.40239579e-08]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.01346971e-09  2.33701588e-10  1.79253090e-10]
 [-7.28169040e-08 -2.59557529e-09 -4.25510107e-08]
 [ 4.50117889e-08 -3.64296522e-09  2.66141849e-08]
 [ 7.25432129e-08  1.64376559e-09  3.82436606e-08]
 [-1.01919426e-07 -3.34007950e-08  1.32364351e-08]
 [-1.09238224e-11 -5.75561094e-12  8.16671824e-12]
 [ 6.87518280e-12  8.63944096e-12 -1.68799229e-11]
 [-2.57741314e-07  7.90609728e-07 -7.78496362e-07]
 [ 1.75519002e-07  1.15975626e-08  7.08353479e-08]
 [-3.96235825e-08  8.79655482e-08 -1.75164989e-07]
 [ 6.37377465e-16 -5.94712453e-16  6.09805263e-16]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-7.27307117e-08  7.77629323e-09 -4.24376984e-08]
 [-4.00681116e-09 -2.17569673e-09  7.37481420e-09]
 [ 1.09970570e-08  1.77759421e-08 -5.52107758e-08]
 [-2.35658637e-10 -3.62805312e-11 -2.99119203e-11]
 [ 1.73855217e-08  3.24544714e-08 -2.89150567e-08]
 [ 1.20937449e-14  7.41749791e-

In [12]:
vis_attributions(attributions_orig, subject, cloud, pred_research_group_orig)

# These two should look identical if the method was correct
vis_attributions(attributions_shuffled, subject, cloud_shuffled, pred_research_group_shuffle)

vis_attributions(attributions_unshuffled, subject, cloud_unshuffled, pred_research_group_shuffle)

Widget(value='<iframe src="http://localhost:34927/index.html?ui=P_0x7f405c56cf50_2&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:34927/index.html?ui=P_0x7f405c5e98b0_3&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:34927/index.html?ui=P_0x7f4047f85e50_4&reconnect=auto" class="pyvi…

Trying explainability with SHAP

In [13]:
# Load model
pickle_pathname = "/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/runs/run_18-03-2025_18-04-04/run_18-03-2025_18-04-04_params.pkl"

with open(pickle_pathname, 'rb') as file:
    
    model_parameters = pickle.load(file)
    
model = model_parameters.model

In [73]:
def pointnet_shap(model, cloud, background, device):

    model.to(device)

    background.to(device)
    
    model.eval()

    # Wrap model in a class as shap needs nn.Module type as input. For some reason pointnet outputs a tuple
    class ModelWrapper(torch.nn.Module):
        def __init__(self, model):

            super(ModelWrapper, self).__init__()
            self.model = model

        def forward(self, x):
            return self.model(x)[0]

    wrapped_model = ModelWrapper(model)

    input = torch.from_numpy(cloud)

    # NN expects float32 on cuda
    input = input.type(torch.float32).to(device)

    # Unsqueeze to add empty batch dimension then transpose  to 3 x n as expected by pointnet
    input = input.unsqueeze(0).transpose(2, 1)
    
    output = wrapped_model(input)
    
    prediction = torch.argmax(torch.nn.functional.softmax(output, dim=1)).cpu().numpy()
    
    mapping = {
        0: 'CN',
        1: 'MCI',
    }
        
    # Get the value of the mapping, -1 if not found
    pred_research_group = mapping.get(int(prediction), -1)

    explainer = shap.DeepExplainer(wrapped_model, background)

    shap_values = explainer.shap_values(input, check_additivity=False)

    # Get only the shap values associated with the prediction
    shap_values = shap_values[:, :, :, prediction]

    # Remove batch dim and transpose back to n x 3
    shap_values = shap_values.squeeze(0)

    shap_values = np.transpose(shap_values)

    print(shap_values.shape)
    
    return shap_values, pred_research_group

In [74]:
cloud = np.load(os.path.join(subject.path, "Left-Hippocampus_aligned_cropped_mesh_downsampledcloud.npy"))

shap_values, pred_research_group = pointnet_shap(model, cloud, background, device)

vis_attributions(shap_values, subject, cloud, pred_research_group)

ValueError: too many values to unpack (expected 3)